(example_Ni(110))=

# Example System: Ni(110) using the ASE interface

In this example we demonstrate how to use the ViPErLEED Python package {term}`tleedm` and the {ref}`atomic simulation environment (ASE) interface<aseapi>` with the example of a Ni(110) surface.

The ViPErLEED API and ASE interface are most relevant to users interested in high-throughput LEED {math}`I(V)` calculations.
We use the example of the Ni(110) surface to showcase how a surface structure can be created, customized in ASE and then passed to ViPErLEED.
In this example, we first create a surface structures for a simple Ni(110)-{math}`(1\times1)` bulk termination.
We then remove one atom to create a {math}`(1\times2)` a missing-row reconstruction and pass om both structures to ViPErLEED.

We show hot to set up a ViPErLEED calculation directly from these ASE atoms objects without writing out a {ref}`POSCAR<poscar>` file.
This yields theoretical LEED {math}`I(V)` curves, which we can easily visualize and between which we can calculate an R-factor.


```{tip}

If this is your first time running ViPErLEED, make sure to first follow the {ref}`installation instructions<installation>`.
For details on how to execute ViPErLEED on your system, see the {ref}`How To Run section<how_to_run>`.

For an example on how to run ViPErLEED without the ASE interface see e.g. {ref}`this tutorial<example_ag_100>`.
```

This example contains Python code to showcase the ASE interface and ViPErLEED API.
To follow along, you can directly copy/past the Python code into a Python environment.
We recommend using a {term}`Jupyter` notebook for this purpose, to directly get visual output.

In [ ]:
# ASE imports - TODO: may not need all
import ase
import ase.build
from ase.visualize.plot import plot_atoms
import ase.io

from viperleed.calc.from_ase import run_from_ase, plot_iv_from_csv
from viperleed.calc.from_ase import rfactor_from_csv, SlabTransform

import matplotlib.pyplot as plt
%matplotlib inline

## Creating the Ni surfaces

The first step is to create the Ni(110) surface structure as an ASE atoms.atoms object.
We can use the convenient surface creations tools contained in the [ase.build module](https://wiki.fysik.dtu.dk/ase/ase/build/surface.html).
Since Ni is an {term}`fcc` metal, we can use the ``ase.build.fcc(110)`` method.
ASE will automatically recognize ``Ni`` as an element label and use the correct experimental lattice constant for the structure.

We first create the bulk-truncated structure with 6 layers.
For our visualization below, we also add 3 Å of vacuum above and below the slab.

In [ ]:
element = 'Ni'
cell_1x1 = ase.build.fcc110(element, size=(1,1,6), vacuum=3)

Using the matploib and the ase.visualize module we can take a look at the atoms object:

In [ ]:
def show_slab_along_z_y(atoms):
    plt.figure()
    fig, axarr = plt.subplots(1, 2, figsize=(10, 5))

    plot_atoms(atoms, axarr[0], rotation=('0x,0y,0z'))
    plot_atoms(atoms, axarr[1], rotation=('-90x,-90y,0z'))

    axarr[0].set_title("View along $z$ (towards bulk)")
    axarr[1].set_title("View along $y$")

    axarr[0].set_xlabel("X-axis, [$\mathrm{\AA}$]")
    axarr[0].set_ylabel("Y-axis, [$\mathrm{\AA}$]")
    axarr[1].set_xlabel("Y-axis, [$\mathrm{\AA}$]")
    axarr[1].set_ylabel("Z-axis, [$\mathrm{\AA}$]")

    fig.show()

In [ ]:
show_slab_along_z_y(cell_1x1)

The Ni atoms are depicted as green spheres and the unit cell (periodic boundary conditions along {math}`$x$` and {math}`$y$`) is indicated with a dashed line.
The surface, as per {ref}`ViPErLEED convention<conventions>` is facing upwards, towards higher {math}`$z$`.

To create the missing row reconstruction, we first make a slab with a {math}`(1\times2)` supercell and 6 layers.
In the atoms object, the atoms are currently sorted from lowest to highest, which we can double check by listing their positions.

In [ ]:
supercell_1x2 = ase.build.fcc110(element, size=(1,2,6), vacuum=3)

# show fractional coordinates of Ni atoms
supercell_1x2.get_scaled_positions()

To create the missing row reconstruction, we thus need to remove one of the two atoms in the surface layer.
We simply copy our supercell and create the missing row reconstruction by removing the last atom:

In [ ]:
missing_row_1x2 = supercell_1x2.copy()
del missing_row_1x2[11] # index starting from 0

show_slab_along_z_y(supercell_1x2) # show the missing row resconstruction
show_slab_along_z_y(missing_row_1x2) # show the missing row resconstruction

## ViPErLEED Input

Now that we have a slab, we can prepare the ViPErLEED calculation.
However, ViPErLEED needs some additional information to run, apart from the input structure.
We generally need to provide multiple input files in a source directory. 
For this example, we create a temporary directory to run the calculation ([Python tempfile module](https://docs.python.org/3/library/tempfile.html)), but you can, of course, also use a persistent path.

In [ ]:
import tempfile
from pathlib import Path

bulk_term_dir = Path(tempfile.mkdtemp())
missing_row_dir = Path(tempfile.mkdtemp())

tempdirs = [bulk_term_dir, missing_row_dir]

### Input Files
The ViPErLEED ASE interface allows you to specify a separate path (argument ``inputs_path``) from which other input files will be copied.
This can be useful for e.g batch processing of test structures with similar settings.

In [ ]:
input_path = "../../_static/example_systems/Ni(110)/"

In our case, we provide two files in the input directory: {ref}`PARAMETERS<parameters>` and `IVBEAMS<ivbeams>`.

#### PARAMETERS

As usual, we provide settings using the {ref}`PARAMETERS file<parameters>`.
In the ViPErLEED ASE interface you can use all parameters available in a typical ViPErLEED run.


```{literalinclude} /_static/example_systems/Ni(110)/PARAMETERS
   :language: console
   :caption: PARAMETERS
```

##### Global Parameters

As usual, we chose the reference calculation by setting {ref}`RUN = 1<run>`.
We can further specify the energy range to calculate and the maximum angular momentum quantum number to use ({ref}`THEO_ENERGIES<theo_energies>` adn {ref}`LMAX<lmax>`).

Normally ViPErLEED would stop execution after the initialization the first time we run with a new structure.
This is to allow the user to double-check the recognized symmetry and the potentially reduced POSCAR file.
For this run via the ASE interface we want to avoid this behavior.
To do so, we can set the {ref}`HALTING parameter = 3<halting>` which means ViPErLEED will not stop unless it encounters a fatal error.

##### Structure Interpretation

We don't have a {ref}`POSCAR file<poscar>`, but we can still use the {ref}`BULK_LIKE_BELOW` parameter, to tell ViPErLEED how to distinguish between the surface and the bulk.
In this example, we can use the bottom two layers as bulk layers, so we can use a value of 0.42 (compare {math}`$z$` positions as shown above).

The ASE interface also has one more feature, that we can use here: By leaving out the {SITE_DEF parameter}, we tell ViPErLEED to use the automatic site detection (for details see {ref}`here<aseapi_auto_sites>`).

##### VIBROCC Parameters
Since we don't provide a {ref}`VIBROCC file<viboccin>`, we need to specify the parameters {ref}`T_DEBYE<t_debye>`, {ref}`T_EXPERIMENT<t_experiment>`, and {ref}`VIBR_AMP_SCALE<VIBR_AMP_SCALE>`.


#### IVBEAMS

Since we want to perform a theory-theory comparison and don't have an {ref}`EXPEBEAMS file<expbeams>`, we also need to provide the file {ref}`IVBEAMS<ivbeams>` which specifies for which diffraction beams to generate output spectra.
Here, we use a very simplistic IVBEAMS with just 4 beams:

```{literalinclude} /_static/example_systems/Ni(110)/IVBEAMS
   :language: console
   :caption: IVBEAMS
```

## Running ViPErLEED

We can now start the ViPErLEED calculation using the ``run_from_ase`` function we imported at the top.
Using ``exec_path`` we specify the directory where to execute the calculation, and with ``inputs_path`` we provide the directory that contains the input files.
The ASE atoms objects is passed via ``ase_object``.
Finally, we need to specify a parameter called ``cut_cell_c_fraction``.
This parameter can be used to discard part of the cell contained in the ASE atoms object by removing any atoms below the provided {math}`$z$` value (fractional coordinates).
This can be useful if you are starting from a symmetric slab, such as commonly used in {term}`DFT` calculations.
In our case, we want to keep the entire slab, so we simply set ``cut_cell_c_fraction=0``.

We use the Jupyter magic command ``%%capture`` to suppress the output of the cell (which would otherwise print out the entire content of the {ref}`ViPErLEED log file<log_files>`).

In [ ]:
%%capture

refcalc_results = run_from_ase(exec_path=bulk_term_dir,
             inputs_path=input_path,
             ase_object=cell_1x1,
             cut_cell_c_fraction=0.0,
             )

theobeams_supercell, _, _ , v0i_bt = refcalc_results

``run_from_ase`` returns 4 values which we store in ``refcalc_results``.
The first 3 objects are string representations of the output files {ref}`THEOBEAMS.csv<theobeams>`, {ref}`Complex_amplitudes_real.csv and Complex_amplitudes_imag.csv<complex_amplitudes_csv>`.
The last return value is a float number giving the value used in the calculation for the imaginary part of the inner potential {math}`V_{0\text{i}}`.
This value could be read from PARAMETERS ({ref}`V0_IMAG<v0_imag>`) and is returned here only for convenience because it is needed for calculating the R-factor later on.

In [ ]:
%%capture

refcalc_results = run_from_ase(exec_path=missing_row_dir,
             inputs_path=input_path,
             ase_object=missing_row_1x2,
             cut_cell_c_fraction=0.0,
             )

theobeams_missing_row, _, _ , v0i_mr = refcalc_results

After we repeat the calculation now also for the supercell with the missing row reconstruction, we now have the theoretical spectra for both surfaces in the form of strings:

In [ ]:
first_10_lines = "\n".join(theobeams_missing_row.split('\n')[:10])
print(first_10_lines)

## Plotting the I(V) curves

We can use the ``plot_iv_from_csv`` function to plot the {math}`$I(V)$` curves for both surfaces to visually compare them:

In [ ]:
# Legends for plot
legends = [f"{element}(110)-($1×1$) Bulk Terminated", f"{element}(110)-($1×2$) Missing Row"]

# call plot_iv_from_csv from ASE interface
figs = plot_iv_from_csv(beam_file=(theobeams_supercell, theobeams_missing_row),
                        beam_file_is_content=(True, True),
                        legends=legends)
plt.show(figs) # show figure

## R-factor calculation

Finally, we can also easily calculate the R-factor between the two curves using the ViPErLEED ASE interface using the ``rfactor_from_csv`` function.
As input we again need to provide the two string representations of ``THEOBEAMS.csv``.
Additionally, we need to specify the value of {math}`V_{0\text{i}}` we received from the calculation above, since it factors into the R-factor calculation (see {ref}`the page on R-factors<r-factor_calculation>` for details).

```{hint}

If you want to compare with a stored {ref}`EXPBEAMS<expbeams>` or {ref}`THEOBEAMS<theobeams>` file, you can also provide the file path rather than the string representation.
In that case, you need to set the value of ``beams_file_is_content`` at the corresponding index to ``False``.
```

In [ ]:
# make sure V0i is same in both cases
assert v0i_bt == v0i_mr
v0i = v0i_bt

In [ ]:
rfactor, v0r = rfactor_from_csv(beams_files=(theobeams_supercell, theobeams_missing_row),
                 v0i=v0i,
                 beams_file_is_content=(True, True))

print(f"R-factor: {rfactor:.4f}")

Even though the {math}`I(V)` curves look qualitatively similar, quantitatively they differ significantly which is represented by the large R-factor. This is, of course, to be expected from two different surface reconstructions!

Finally, as a last step, let's not forget to clean up the temporary directories created before:

In [ ]:
import shutil
#cleanup
for t_dir in tempdirs:
    shutil.rmtree(t_dir)